In [ ]:
%cd /content/drive/MyDrive/5팀(윤정준, 최애림, 진청아, 이정현a)/코드

/content/drive/.shortcut-targets-by-id/1niQkF2oFYkuV2aahMfHbpa3kXs5yrcAd/5팀(윤정준, 최애림, 진청아, 이정현a)/코드


# mask 이미지, mask 이미지에서 원본 부분 패치화해서 저장

# 필요 라이브러리 설치

In [ ]:
# openslide, staintools, spams 라이브러리 설치
!pip install openslide-python
!apt-get install openslide-tools
!apt-get install python3-openslide
!pip install ipython-autotime
!pip install staintools
!pip install spams

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 338 kB 31.6 MB/s 
  Created wheel for openslide-python: filename=openslide_python-1.2.0-cp38-cp38-linux_x86_64.whl size=27711 sha256=503d28819302a038be419374096d2ac2e7df8d01698967f5714deef6375c1526
  Stored in directory: /root/.cache/pip/wheels/ae/74/4e/f8654d229eb249d1345e3df0191030ad77e7a6a2114df7cd28
Successfully built openslide-python
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 7 not upgraded.
Need to get 92.5 kB of archives.
After this

# import

In [ ]:
import os
import pickle

import numpy as np
import pandas as pd
import openslide
import staintools

from glob import glob
from datetime import datetime
from PIL import Image
from tqdm import tqdm_notebook as tqdm
from openslide.deepzoom import DeepZoomGenerator
from skimage.filters import threshold_otsu, threshold_yen

import warnings
warnings.filterwarnings("ignore")

from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img, array_to_img

# 경로설정

## 1) 저장할 폴더 생성
 - 패치화한 이미지, 좌표저장 pickle 형태

In [ ]:
# 패치 저장할 폴더 생성 
dir_path = './patches' 
img_mask_pairs_path = './patches/img_mask_pairs' # 원본이미지, 마스킹된이미지 좌표저장 폴더명

## 폴더가 없으면 만들고 있으면 넘어가기
if not os.path.exists(f'{dir_path}/'):
  os.mkdir(f'{dir_path}/')
if not os.path.exists(f'{img_mask_pairs_path}/'):
  os.mkdir(f'{img_mask_pairs_path}/')
if not os.path.exists(f'{img_mask_pairs_path}/개별pk/'):
  os.mkdir(f'{img_mask_pairs_path}/개별pk/')
else:
  if not os.path.exists(f'{dir_path}/train/'): # 패치화된 원본사진
      os.mkdir(f'{dir_path}/train/')        
  if not os.path.exists(f'{dir_path}/mask/'): # 패치화된 마스킹사진
      os.mkdir(f'{dir_path}/mask/')

In [ ]:
# 마스킹 된 사진만 패치화하기
mask_base_dir = './train_masks/'
slide_dir = './train_imgs/'
mask_tumor_dir = './patches/whole_mask_tumor/'

filenames_mask = os.listdir(mask_base_dir)

train_path_list = [slide_dir + filename for filename in filenames_mask]
mask_path_list_tumor = [mask_tumor_dir + filename for filename in filenames_mask]

In [ ]:
# 최종 저장 경로
patches_img_path = './patches/train'
patches_mask_path = './patches/mask'
img_mask_pairs_path = './patches/img_mask_pairs'

# 설정 패치 사이즈
patch_size=256

##2) 좌표 추출

### 이미지 좌표 데이터프레임으로 저장
 - 조직이 있는 부분, 암이 있는 부분 

In [ ]:
def find_patches_from_slide( slide_path, mask_path):
# for i, filename in enumerate(train_path_list):
  # train _ whole 이미지
    with openslide.open_slide(slide_path) as slide:
        tiles = DeepZoomGenerator(slide, tile_size=256, overlap=0, limit_bounds=False)
        size = tiles.level_tiles[tiles.level_count-1]
        thumb_slide = slide.get_thumbnail(size)

    # tumor 부분
    with openslide.open_slide(mask_path) as mask:
        thumb_mask = mask.get_thumbnail(size)

    # ############## 조직인 부분 ##############
    slide4_grey = np.array(thumb_slide.convert('L'))
    binary = slide4_grey < 255  # white = 255 
    slide4_not_white = slide4_grey[binary] 
    thresh = threshold_otsu(slide4_not_white) # otsu 사용

    height, width = slide4_grey.shape  # (height, width)
    for h in range(height):
        for w in range(width):
            if slide4_grey[h, w] > thresh: 
                binary[h, w] = False
                
    # 패치된 부분 데이터프레임 생성
    patches = pd.DataFrame(pd.DataFrame(binary).stack())
    patches['is_tissue'] = patches[0]
    patches = pd.DataFrame(pd.DataFrame(binary).stack(), columns=['is_tissue'])

    # ############## tumor인 부분 ##############
    slide4_grey_mask = np.array(thumb_mask.convert('L'))
    binary_mask = slide4_grey_mask < 255
    slide4_grey_mask[binary_mask] # white = 255 배경제거
    slide4_not_white_mask = slide4_grey_mask[binary_mask]
    thresh_mask = threshold_otsu(slide4_not_white_mask) # otsu 사용

    # print(f'current thersh : {thresh}')

    height, width = slide4_grey_mask.shape
    for h in range(height):
      for w in range(width):
        if slide4_grey_mask[h, w] > thresh_mask:
          binary_mask[h, w] = False

    ## 암인 부분의 데이터 프레임 생성 후 위에서 저장한 조직인부분의 데이터프레임과 concat
    patches_y = pd.DataFrame(pd.DataFrame(binary_mask).stack(), columns=['is_tumor'])
    samples = pd.concat([patches, patches_y], axis=1)

    # 조직인부분만 저장 (normal인 부분 제외)
    samples = samples[samples['is_tissue']==True]
    # 좌표 저장
    samples['tile_loc'] = samples.index.tolist()
    samples.reset_index(inplace=True, drop=True)

    return samples #(is_tissue, is_tumor, tile_loc)

## 3) 이미지 저장

### mask된 부분, 암인 부분 패치화로 저장

- 경로 설정 (딕셔너리 형태)

In [ ]:
# 원본이미지 경로, 암인 부분 이미지 경로 각각 출력
slide_path_dict, mask_path_dict = {}, {}
for slide_path, mask_path in zip(train_path_list, mask_path_list_tumor):

  #slide
    slide_name, _ = os.path.splitext(slide_path)
    slide_idx = slide_name.split('/')[-1] 
    
    #mask
    mask_name, _ = os.path.splitext(mask_path)
    mask_idx = mask_name.split('/')[-1]
    
    # update each dictionary
    slide_path_dict[slide_idx] = slide_path
    mask_path_dict[mask_idx] = mask_path

- 이미지 저장

In [ ]:
# 58장의 이미지 이름 : 이미지 저장 경로
slide_mask_path_pairs = [(idx, slide_path, mask_path_dict[idx]) for idx, slide_path in slide_path_dict.items()]

# 패치 저장하기 
slide_mask_whole_pairs = []
for cnt, (s_idx, slide_path, mask_path) in enumerate(slide_mask_path_pairs):
  # 위에서 만든 쪼개논 패치중 조직인 부분과 암인 부분 가져오기 (true, false_데이터프레임형태)
  samples = find_patches_from_slide(slide_path, mask_path)

  # 각 패치된 이미지 저장경로
  slide_mask_pairs = []

  # id 별로 이미지 패치화
  with openslide.open_slide(slide_path) as slide:
    with openslide.open_slide(mask_path) as mask:
      slide_tiles = DeepZoomGenerator(slide, tile_size=patch_size, overlap=0, limit_bounds=False)
      mask_tiles = DeepZoomGenerator(mask, tile_size=patch_size, overlap=0, limit_bounds=False)

      # 조직이라고 저장되어있는 부분 좌표불러오기
      for p_idx, tile_loc in enumerate(samples['tile_loc'].tolist()):
        y, x = tile_loc

        try:
          # 각 좌표별로 train, mask 이미지 타일 불러오기
          img = slide_tiles.get_tile(slide_tiles.level_count-1, (x, y))
          mask_img = mask_tiles.get_tile(mask_tiles.level_count-1, (x, y))

        except:
          pass

       # 사이즈 확인 후 저장
        if img.size == (256, 256):
          patche_train_path = f'{patches_img_path}/{s_idx}/{s_idx}_{p_idx}'
        if mask_img.size == (256, 256):
          patche_mask_path = f'{patches_mask_path}/{s_idx}/{s_idx}_{p_idx}'
        else:
          pass
        ## 사이즈 모자란 부분 확인
          # print('사이즈 부족으로 pass')
          # print(f'{s_idx}_{p_idx} 이미지 사이즈 : img({img.size}), mask({mask_img.size})')
        
        # mask 이미지 바탕인 부분 저장하지 않기 
        array = img_to_array(mask_img)
        myset = set(array.reshape(-1))
        if [x for x in myset] == [255.0]:
          patche_mask_path = '-'
        
        # 패치 번호순으로 사진 경로 저장 (원본이미지, 암인부분이미지)
        slide_mask_pairs.append((f'{patche_train_path}.png',
                                                    f'{patche_mask_path}.png'))
        
        slide_mask_whole_pairs.append((f'{patche_train_path}.png',
                                                                f'{patche_mask_path}.png'))
        
        if not os.path.exists(f'{patches_mask_path}/{s_idx}'):
          os.mkdir(f'{patches_mask_path}/{s_idx}')
        if not os.path.exists(f'{patches_img_path}/{s_idx}'):
          os.mkdir(f'{patches_img_path}/{s_idx}')

        # 이미지 저장
        img.save(f'{patche_train_path}.png')

        # mask 흰바탕 제거
        if patche_mask_path == '-':
          pass
        else:
          mask_img.save(f'{patche_mask_path}.png')
  
  if cnt % 5 == 0:
    with open(f'{img_mask_pairs_path}/개별pk/img_mask_pairs_{s_idx}_{cnt}.pkl', 'wb') as f:
      pickle.dump(slide_mask_pairs, f)

# 전체 패치 경로 저장
with open(f'{img_mask_pairs_path}/img_mask_whole_pairs.pkl', 'wb') as f:
  pickle.dump(slide_mask_whole_pairs, f)